In [1]:
import pandas as pd
import numpy as np
import math

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from datetime import datetime
from datetime import timedelta

# Merge all flight data

In [2]:
cf_start_year = 2010
cf_end_year = 2023
cf_year_range = [y for y in range(cf_start_year, cf_end_year+1) if y != 2020]

cf_path_folder = "full_historical_data_bf2024"
cf_path_list = [cf_path_folder + 
                "/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_" + 
                f"{j}" + "_" + f"{i}" + ".csv" for i in range(1,13) for j in cf_year_range]

cf_start = datetime(cf_start_year, 1, 1)
cf_end = datetime(cf_end_year, 12, 31)

In [3]:
# gets all files and puts them into a dict
df_dict = {}
for i, file_path in enumerate(cf_path_list):
    df_dict[f"df_{i+1}"] = pd.read_csv(file_path)

C:\Users\matth\AppData\Local\Temp\ipykernel_9132\3286583175.py:4: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[f"df_{i+1}"] = pd.read_csv(file_path)
C:\Users\matth\AppData\Local\Temp\ipykernel_9132\3286583175.py:4: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[f"df_{i+1}"] = pd.read_csv(file_path)
C:\Users\matth\AppData\Local\Temp\ipykernel_9132\3286583175.py:4: DtypeWarning: Columns (76,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[f"df_{i+1}"] = pd.read_csv(file_path)
C:\Users\matth\AppData\Local\Temp\ipykernel_9132\3286583175.py:4: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[f"df_{i+1}"] = pd.read_csv(file_path)
C:\Users\matth\AppData\Local\Temp\ipykernel_9132\3286583175.py:4: DtypeWarning: Columns (77,84) have mixed types. Specify 

In [4]:
print(sum(len(df) for df in df_dict.values()))
print(df_dict['df_1'].columns)

82133464
Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'DOT_ID_Reporting_Airline',
       'IATA_CODE_Reporting_Airline', 'Tail_Number',
       ...
       'Div4TailNum', 'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID',
       'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff',
       'Div5TailNum', 'Unnamed: 109'],
      dtype='object', length=110)


## Configure columns 

In [5]:
columns_to_keep = ['FlightDate', 'OriginAirportID', 'DestAirportID', 'CRSDepTime', 'WeatherDelay', 'Cancelled', 'CancellationCode', 'Reporting_Airline']

df_test = pd.concat([df[columns_to_keep] for df in df_dict.values()], ignore_index=True)

### Reduce memory of df

In [8]:
df_test.dtypes

FlightDate            object
OriginAirportID        int64
DestAirportID          int64
CRSDepTime           float64
WeatherDelay         float64
Cancelled            float64
CancellationCode      object
Reporting_Airline     object
dtype: object

In [9]:
# Convert Cancelled to bool
df_test['Cancelled'] = df_test['Cancelled'].astype('boolean')
# Downcast floats to float32
float_cols = ['CRSDepTime', 'WeatherDelay', 'CancelledWeather']
for col in float_cols:
    if col in df_test.columns:
        df_test[col] = pd.to_numeric(df_test[col], downcast='float')

### Add Cancelled due to Weather column

In [10]:
df_test["CancelledWeather"] = df_test["Cancelled"].where(df_test["CancellationCode"] == "B", False)

### Merge with lookup tables to get standard airport codes

In [11]:
# get lookup tables for airport_id and airport codes
airport_ids = pd.read_csv('misc/L_AIRPORT_ID.csv')
airport_codes = pd.read_csv('misc/L_AIRPORT_CODES.csv')

In [12]:
# Step 1: Map airport ID -> Name
id_to_name = dict(zip(airport_ids['Code'], airport_ids['Description']))

# Step 2: Map Name -> Code
name_to_code = dict(zip(airport_codes['Description'], airport_codes['Code']))

# Step 3: Apply both mappings
df_test['Origin_Airport'] = df_test['OriginAirportID'].map(id_to_name).map(name_to_code)

In [13]:
df_test = df_test.drop(columns=['OriginAirportID', 'DestAirportID'])

### Format time data 

In [14]:
df_test.head()

,FlightDate,CRSDepTime,WeatherDelay,Cancelled,CancellationCode,Reporting_Airline,CancelledWeather,Origin_Airport
0,2010-01-09,1056.0,0.0,False,NaN,9E,False,ATL
1,2010-01-16,1056.0,0.0,False,NaN,9E,False,ATL
2,2010-01-23,1056.0,NaN,False,NaN,9E,False,ATL
3,2010-01-30,1056.0,NaN,False,NaN,9E,False,ATL
4,2010-01-05,1755.0,NaN,False,NaN,9E,False,ATL


In [15]:
df_test['date'] = pd.to_datetime(df_test['FlightDate'])

In [16]:
# one row has an NaN value
df_test = df_test.dropna(subset=["CRSDepTime"])

In [17]:
# pad time with zeros, then combine into one datetime column
# NOTE: CRSDepTime is local time
df_test["datetime"] = df_test["date"] + pd.to_timedelta(
    df_test["CRSDepTime"].apply(lambda x: int(x // 100) * 60 + int(x % 100)), unit="m"
)
df_test.head()

,FlightDate,CRSDepTime,WeatherDelay,Cancelled,CancellationCode,Reporting_Airline,CancelledWeather,Origin_Airport,date,datetime
0,2010-01-09,1056.0,0.0,False,NaN,9E,False,ATL,2010-01-09,2010-01-09 10:56:00
1,2010-01-16,1056.0,0.0,False,NaN,9E,False,ATL,2010-01-16,2010-01-16 10:56:00
2,2010-01-23,1056.0,NaN,False,NaN,9E,False,ATL,2010-01-23,2010-01-23 10:56:00
3,2010-01-30,1056.0,NaN,False,NaN,9E,False,ATL,2010-01-30,2010-01-30 10:56:00
4,2010-01-05,1755.0,NaN,False,NaN,9E,False,ATL,2010-01-05,2010-01-05 17:55:00


In [18]:
df_test['WeatherDelay'] = df_test['WeatherDelay'].fillna(0)

df_test.head()

,FlightDate,CRSDepTime,WeatherDelay,Cancelled,CancellationCode,Reporting_Airline,CancelledWeather,Origin_Airport,date,datetime
0,2010-01-09,1056.0,0.0,False,NaN,9E,False,ATL,2010-01-09,2010-01-09 10:56:00
1,2010-01-16,1056.0,0.0,False,NaN,9E,False,ATL,2010-01-16,2010-01-16 10:56:00
2,2010-01-23,1056.0,0.0,False,NaN,9E,False,ATL,2010-01-23,2010-01-23 10:56:00
3,2010-01-30,1056.0,0.0,False,NaN,9E,False,ATL,2010-01-30,2010-01-30 10:56:00
4,2010-01-05,1755.0,0.0,False,NaN,9E,False,ATL,2010-01-05,2010-01-05 17:55:00


# Filter by Airport

In [19]:
airports = [
    "ATL", "ORD", "JFK", "LGA", "LAX",
    "SFO", "IAH", "HOU", "DEN", "MCO",
    "MIA", "LAS", "IAD", "PHX", "SEA",
    "BOS", "CLT", "ANC", "PDX",
    
    "MSP", "SLC", "BOI", "JAC", "FSD",
    "STL", "BZN", "ABQ", "OKC", "DSM"
]

In [20]:
df_test = df_test[df_test['Origin_Airport'].isin(airports)]

In [21]:
df_test['datetime'] = pd.to_datetime(df_test['datetime'], errors='coerce', unit='m')

In [22]:
df_test = df_test.sort_values(by=["Origin_Airport", "datetime"])

In [23]:
df_test.to_csv("final_extra_airports_2010_to_2023.csv", index=False)

# Get Weather Data in separate df
### Get snapshots at 2-hour intervals at all airports

In [3]:
from meteostat import Stations, Daily, Point, Hourly
import pandas as pd

stations = Stations()

all_stations = stations.region('US').fetch()

In [4]:
airports = [
    "ATL", "ORD", "JFK", "LGA", "LAX",
    "SFO", "IAH", "HOU", "DEN", "MCO",
    "MIA", "LAS", "IAD", "PHX", "SEA",
    "BOS", "CLT", "ANC", "PDX",
    
    "MSP", "SLC", "BOI", "JAC", "FSD",
    "STL", "BZN", "ABQ", "OKC", "DSM"
]

cf_start = pd.to_datetime('2010-1-1')
cf_end = pd.to_datetime('2023-12-31')

In [5]:
"""Find a Meteostat station for an IATA code."""
def find_station_for_code(df_stations, code):
    if 'iata' in df_stations.columns:
        s = df_stations[df_stations['iata'] == code]
        if not s.empty:
            return s.iloc[0]
    if 'icao' in df_stations.columns:
        # exception: ANC is PANC 
        if (code == "ANC"):
            s = df_stations[df_stations['icao'] == f"P{code}"]
        else:
            s = df_stations[df_stations['icao'] == f"K{code}"]
        if not s.empty:
            return s.iloc[0]
        s = df_stations[df_stations['icao'] == code]
        if not s.empty:
            return s.iloc[0]
    return None

weather_snapshots = []

for airport in airports:
    try:
        st = find_station_for_code(all_stations, airport)
        if st is None:
            print(f"No station found for {airport}, skipping.")
            continue

        loc = Point(st['latitude'], st['longitude'], st['elevation'])

        # Fetch hourly data
        data = Hourly(loc, cf_start, cf_end).fetch()

        if data.empty:
            print(f"No weather data for {airport}.")
            continue

        # Convert to DataFrame with datetime
        data.reset_index(inplace=True)
        data['Origin_Airport'] = airport

        # Keep exact 2-hour snapshots (no averaging)
        data['time'] = pd.to_datetime(data['time'])
        data = data.set_index('time')
        
        # Keep only rows at exact 2-hour timestamps
        data = data[data.index.minute == 0]          # ensure on-the-hour
        data = data[data.index.hour % 2 == 0]        # every 2 hours (0, 2, 4, 6...)
        
        data = data.reset_index()

        # Rename for clarity
        data.rename(columns={
            'time': 'datetime',
            'temp': 'temperature_C',
            'dwpt': 'dew_point_C',
            'rhum': 'humidity_percent',
            'prcp': 'precipitation_mm',
            # 'snow': 'snowfall_mm',
            'wdir': 'wind_direction_deg',
            'wspd': 'wind_speed_kmh',
            'wpgt': 'wind_gust_kmh',
            'pres': 'pressure_hPa',
            'tsun': 'sunshine_minutes',
            'coco': 'weather_code'
        }, inplace=True)

        weather_snapshots.append(data)
        print(f"{airport} complete, {len(data)} rows")

    except Exception as e:
        print(f"=Error for {airport}: {e}")

# Combine all airports into one DataFrame
weather_all = pd.concat(weather_snapshots, ignore_index=True)

ATL complete, 61345 rows


ORD complete, 61345 rows


JFK complete, 61345 rows


LGA complete, 61345 rows
LAX complete, 61345 rows


SFO complete, 61345 rows


...

JAC complete, 61345 rows
FSD complete, 61345 rows


STL complete, 61345 rows


BZN complete, 61345 rows


ABQ complete, 61345 rows
OKC complete, 61345 rows


DSM complete, 61345 rows


In [6]:
weather_all.head()

,datetime,temperature_C,dew_point_C,humidity_percent,precipitation_mm,snow,wind_direction_deg,wind_speed_kmh,wind_gust_kmh,pressure_hPa,sunshine_minutes,weather_code,Origin_Airport
0,2010-01-01 00:00:00,7.2,6.8,97.0,<NA>,<NA>,290.0,5.4,<NA>,1018.9,<NA>,<NA>,ATL
1,2010-01-01 02:00:00,7.2,6.8,97.0,0.0,<NA>,280.0,7.6,<NA>,1019.6,<NA>,<NA>,ATL
2,2010-01-01 04:00:00,6.7,6.7,100.0,0.0,<NA>,270.0,5.4,<NA>,1019.5,<NA>,<NA>,ATL
3,2010-01-01 06:00:00,6.7,6.7,100.0,0.0,<NA>,270.0,7.6,<NA>,1019.1,<NA>,<NA>,ATL
4,2010-01-01 08:00:00,6.7,5.6,93.0,0.0,<NA>,330.0,16.6,<NA>,1018.8,<NA>,<NA>,ATL


In [19]:
weather_all.columns

Index(['datetime', 'temperature_C', 'dew_point_C', 'humidity_percent',
       'precipitation_mm', 'snow', 'wind_direction_deg', 'wind_speed_kmh',
       'wind_gust_kmh', 'pressure_hPa', 'sunshine_minutes', 'weather_code',
       'Origin_Airport'],
      dtype='object')

In [7]:
weather_all.to_csv("weather_extra_airports_2010_to_2023.csv")